In [1]:
import os
%pwd

'd:\\Final-Year-Project\\Credit-Card-Fraud-Detection-Using-GNN\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Final-Year-Project\\Credit-Card-Fraud-Detection-Using-GNN'

In [3]:
# Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [5]:
# Configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir) 
        )

        return data_ingestion_config

In [6]:
import os
import zipfile
import subprocess
from Credit_Card_Fraud_Detection import logger

In [7]:
# Components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not self.config.local_data_file.exists():
            try:
                # Use gdown with --fuzzy for flexible link handling
                subprocess.run([
                    "gdown",
                    "--fuzzy",
                    self.config.source_URL,
                    "-O",
                    str(self.config.local_data_file)
                ], check=True)
                logger.info(f"Downloaded file to {self.config.local_data_file}")
            except subprocess.CalledProcessError as e:
                logger.error(f"Failed to download file: {e}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}, size: {get_size(self.config.local_data_file)}")



    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted ZIP file to {self.config.unzip_dir}")
        except zipfile.BadZipFile:
            logger.error("The downloaded file is not a valid ZIP file.")
        except FileNotFoundError:
            logger.error("ZIP file not found.")
  

In [8]:
# Pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-24 09:29:31,839: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-24 09:29:31,843: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-24 09:29:31,846: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-24 09:29:31,847: INFO: common: created directory at: artifacts]
[2025-03-24 09:29:31,847: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-24 09:30:26,467: INFO: 213065244: Downloaded file to artifacts\data_ingestion\data.zip]
[2025-03-24 09:30:27,892: INFO: 213065244: Extracted ZIP file to artifacts\data_ingestion]
